# Data Analysis

In [1]:
import tensorflow as tf

In [2]:
from datasets import load_dataset
raw_dataset = load_dataset("squad")

c:\Users\acayt\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad (C:/Users/acayt/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 16.66it/s]


In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [4]:
raw_dataset.shape

{'train': (87599, 5), 'validation': (10570, 5)}

In [5]:
print("ID: ", raw_dataset['train'][0]['id'])
print("Title: ", raw_dataset['train'][0]['title'])
print("Context: ", raw_dataset['train'][0]['context'])
print("Question: ", raw_dataset['train'][0]['question'])
print("Answer: ", raw_dataset['train'][0]['answers'])

ID:  5733be284776f41900661182
Title:  University_of_Notre_Dame
Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


*Attributes `ID` and `Title` are not useful.*

In [6]:
raw_dataset['train'].filter(lambda x: len(x['answers']['text']) != 1)

Loading cached processed dataset at C:/Users/acayt/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-6f50cc0ea6e240f9.arrow


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

*During training, there is only one possible answer. This is a double-check mechanism put in place.\
Evaluation set may contain multiple answers, but that's not of concern.*

# Preprocessing

In [7]:
from transformers import AutoTokenizer

In [8]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
tokenizer.is_fast

True

*A check to make sure that fast-tokenizer is implemented in the used model type (BERT).*

In [11]:
context = raw_dataset['train'][0]['context']
question = raw_dataset['train'][0]['question']

inputs = tokenizer(question, context)
tokenizer.decode(inputs['input_ids'])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

*The format seems like `[CLS]` question `[SEP]` context `[SEP]`.*

*Using Sliding window to deal with long features by splitting them.*

In [13]:
inputs = tokenizer(
    question,
    context,
    max_length=100, #length limit
    truncation="only_second", #to only split the 'Context'
    stride=50, #no of overlapping tokens between 2 successive windows
    return_overflowing_tokens=True, #to consider the overflowing tokens
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the B

*Context splitted into 4 windows. Answer appears in 3rd and 4th windows.*